In [1]:
from pathlib import Path

from datasets import load_dataset

## Dataset

In [2]:
dataset_id = "google_wellformed_query"
dataset = load_dataset(path=dataset_id)

Generating train split:   0%|          | 0/17500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3850 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3750 [00:00<?, ? examples/s]

In [4]:
dataset["train"]["content"][:2]

['The European Union includes how many ?',
 'What are Mia Hamms accomplishment ?']

## Language Model

### Probabilistic Language Modeling
- 🎯 **Goal** - compute the probability of a sentence or sequence of words in our case:
$$ P(W) = (W_1, W_2, W_3, W_4, W_5 ... W_n) $$ 
- 🖇️ **Related task** - probability of an upcoming word:
$$ P(W_5|W_1, W_2, W_3, W_4) $$

Basically these model is telling us how well this words fit together.

### How to compute P(W)
- How to compute this joint probability:
$$ P(its, water, is, so, transparent, that) $$
- Intuition: let's rely on the `Chain Rule of Probability`

#### Reminder: The Chain Rule
- Recall the definition of conditional probabilities
$$ P(A|B) = {P(A,B) \over P(B)} $$
$$ P(A|B)P(B) = P(A,B) $$
$$ P(A,B) = P(A|B)P(B)$$
- The Chain Rule in General
$$ P(x_1, x_2, x_3, ... , x_n) = P(x_1)P(x_2|x_1)P(x_3|x_1, x_2) ... P(x_n|x_1, ... ,x_{n-1}) $$

### The Chain Rule applied to compute joint probability of words in sentence
$$ P(\text{"its water is so transparent that"}) = $$
$$ P(its)P(water| its)P(is|its water)P(so|its water is)P(transparent|its water is so)P(that|its water is so transparent) $$


$$ P(w_1, w_2 ... w_n) = \prod_i P(w_i|w_1, w_2 ... w_{i-1}) $$

### Markov Assumption by Andrei Markov
- Simplifying assumption:
$$ P(the|\text{its water is so transparent that}) \approx P(the|that) $$
- Or maybe:
$$ P(the|\text{its water is so transparent that}) \approx P(the|\text{that transparent}) $$
- More formally:
$$ P(w_1, w_2 ... w_n) \approx \prod_i P(w_i|w_{i-k} ... w_{i-1}) $$
The probability of sequence of words is the product of conditional probability of that word given some prefix of last few words.
In other words, we approximate each component in the product
$$ P(w_i|w_1w_2 ... w_{i-1}) \approx P(w_i|w_{i-k} ... w_{i-1}) $$

#### Simplest case: Unigram model
$$ P(w_1, w_2 ... w_n) \approx \prod_i P(w_i) $$

Some automatically generated sentences from a unigram model:

thrift, did, eighty, said, hard, 'm ...

We would get random sequence of words that woud not look anything like sentences.

#### Bigram model
- Condition on the previous word:
$$ P(w_i|w_1w_2 ... w_{i-1}) \approx P(w_i|w_{i-1}) $$

Some automatically generated sentences from a unigram model:

outside, new, car, parking, lot, of, the, agreement ...

### N-gram models

- We can extend to trigrams, 4-grams, 5-grams
- In general this is an insufficient model of language
    - because language has <span style="color:green;">long-distance dependencies</span>:<br>
    "The computer which I had just put into the machine room on the fifth floor crashed."
- But we can often get away with N-gram models